13-6. 모델을 더 완벽하게 만드는 방법 : KerasTuner

우리가 볼 첫번째 MLOps는 하이퍼파라미터 튜닝입니다. 그중에서 우리가 사용할 툴은 KerasTuner입니다.

KerasTuner는 Keras뿐만 아니라 scikit-learn과 같은 모델에서도 커스터마이징해서 사용할 수 있는 툴이며 하이퍼파라미터 튜닝을 자동으로 할 수 있게 도와줍니다.

기존 모델에서 하이퍼파라미터 튜닝을 진행한다고 했을 때 딥러닝의 경우 일일이 바꿔가면서 함수를 만들고 정리했습니다. 그러나 KerasTuenr를 사용하고 범위를 결정하는 함수를 잘 선택한다면 하이퍼파라미터 튜닝을 원하는대로 할 수 있습니다.

그렇다면 KerasTuner를 사용하러 떠나볼까요?

이번 실습은 MNIST로 간단하게 할 수 있는 하이퍼파라미터 튜닝작업입니다!

우선 디렉토리 먼저 만들어놓도록 하겠습니다.

이번 실습에 앞서 우선 KerasTuner를 설치하겠습니다.

In [1]:
!mkdir ~/aiffel/mlops
!pip install keras-tuner

     |████████████████████████████████| 129 kB 6.8 MB/s            


In [3]:
import tensorflow as tf
import keras
import keras_tuner as kt
from sklearn.model_selection import train_test_split
import os

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

CNN을 사용할 예정이라 차원 수를 하나 더 추가해줍니다.
또한 label을 categorical을 활용해 변환합니다.

In [5]:
X_train = x_train.reshape(-1,28, 28, 1) 
X_test = x_test.reshape(-1,28,28,1)
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

scikit-learn에 내장되어 있는 train_test_split으로 train data와 validation data를 나누어줍니다.

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)
print(X_train.shape)

(48000, 28, 28, 1)


이번에는 제가 짠 DeepTuner를 살펴보겠습니다!

DeepTuner의 경우 kerastuner.Tuner를 인자로 하는 class이며 class에서 수행하는 함수는 run_trial, save_model load_model입니다.

run_trial 함수에서 제일 중요한 부분은 hypermodel과 trial입니다.

KerasTuner에서의 hypermodel은 모델을 공유 및 재사용하기 위해 검색 공간을 캡슐화하는 모델입니다. hypermodel의 경우 hp라는 인수를 활용해서 keras.Model을 생성합니다.
즉 hypermodel은 우리가 만들고 싶은 모델을 쌓는 과정을 거치는데 이때 하이퍼파라미터 튜닝에 대한 검색공간을 만들어줄때 hp라는 인수를 사용해서 만든 모델입니다.
hypermodel의 경우 build 메소드를 활용하면 모델이 빌드가 되면서 하이퍼파라미터 튜닝이 시작합니다.

trial의 경우에는 Oracle에 속하는 class입니다.
Oracle이란 KerasTuner의 모든 검색 알고리즘에서 사용하는 기본 클래스이며 크게 RandomSearchOracle, BayesianOptimizationOracle, HyperbandOracle이 있습니다.
쉽게 설명하면 Oracle은 KerasTuner가 하이퍼파라미터를 정할 때 사용하는 알고리즘이라고 생각하시면 됩니다!
여기서 trial.hyperparameter는 Oracle이 찾아야 하는 하이퍼파라미터입니다. 즉 hypermodel에서의 hp입니다.
제가 model.fit()을 할때 batch_size도 고를 수 있게 만들었습니다.

In [7]:
class DeepTuner(kt.Tuner):
    def run_trial(self, trial, X, y, validation_data, **fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)
        model.fit(X, y, batch_size=trial.hyperparameters.Choice(
            'batch_size', [16, 32]), **fit_kwargs)


        X_val, y_val = validation_data
        eval_scores = model.evaluate(X_val, y_val)
        return {name: value for name, value in zip(
            model.metrics_names,
            eval_scores)}

이번에는 build_model쪽을 보도록 하겠습니다.

build_model은 위에서 설명한것처럼 hypermodel을 만들어줘야 합니다. 제가 만든 hypermodel은 총 2가지 기법이 들어가 있으며 주의사항도 1가지 있습니다.

우선 주의사항부터 말씀드리면 해당 모델의 경우 hypermodel이기 때문에 Input지정이 필수입니다!
그렇기에 여러분들이 넣고 싶은 모델에 대한 shape을 꼭 기재해주셔야 합니다!

제가 사용한 첫번째 기법은 바로 layer의 숫자도 KerasTuner에게 맡겼습니다.
for문을 확인해보겠습니다

첫번째 for문의 경우 hp.Int로 만들어 검색공간은 정수로 만들고 가장 작은값을 1로 가장 큰값을 10으로 두었습니다.
이렇게 설정하면 최소 1개에서 최소 10개의 layer를 쌓을 수 있게 설정할 수 있습니다.

제가 쌓고싶은 layer는 conv2D인데 kernel_size는 (3,3)이며 차원수는 최소 32에서 최대 256으로 바꾸었습니다.

두번째 for문을 살펴보겠습니다. 두번째 for문도 최소 1개에서 3개로 설정했지만 Dense Layer의 경우 나올 수 있는 차원을 32,64,128,256중 1개를 선택하도록 만들었습니다.

이러한 방식으로 hypermodel을 만들면 하고싶은 하이퍼 파라미터 튜닝을 진행할 수 있습니다.

In [8]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape = X_train.shape[1:], name = 'inputs'))
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
              model.add(tf.keras.layers.Conv2D(hp.Int(
                  'units_{i}'.format(i=i), min_value=32, max_value=128, step=5), (3,3),activation='relu'))
    model.add(tf.keras.layers.Flatten())
    for i in range(hp.Int('n_connections', 1, 3)):
        model.add(tf.keras.layers.Dense(hp.Choice(f'n_nodes',
                                  values=[32,64,128, 256]), activation = 'relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax', name = 'outputs'))
    model.compile(optimizer = 'adam',loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

마지막으로 keras_tuner를 정의하고 탐색하는것까지 보도록 하겠습니다.

저는 이번 모델의 경우 BayesianOptimizationOracle을 사용할 예정이며 목표는 accuracy와 max로 둘 예정입니다. 실제 trial은 10번으로 지정할 것입니다.

hypermodel은 build_model을 넣어주시고 project이름도 작성해주세요.

마지막으로 search함수에 X_train, Y_train, validation data, epoch을 넣고 탐색합니다!

In [9]:
my_keras_tuner = DeepTuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('accuracy', 'max'),
        max_trials=10,
        seed=42),
    hypermodel=build_model,
    overwrite=True,
    project_name='my_keras_tuner')

# 해당 모델 학습시간은 약 10분정도 걸립니다!
my_keras_tuner.search(
    X_train, y_train, validation_data=(X_val, y_val), epochs=3)

Trial 10 Complete [00h 01m 20s]
accuracy: 0.9665833115577698

Best accuracy So Far: 0.9868333339691162
Total elapsed time: 00h 12m 28s


가장 좋은 모델을 뽑는 방법은 KerasTuner.get_best_hyperparamters를 이용해서 가장 좋은 하이퍼파라미터를 뽑아내는 작업입니다
하이퍼파라미터를 뽑았으면 build_model()에 집어넣어 가장 좋은 모델을 선언합니다.

그렇다면 여러분들이 만든 가장 좋은 모델을 확인해볼까요?

In [10]:
best_hps = my_keras_tuner.get_best_hyperparameters(num_trials=10)[0]
model = build_model(best_hps)
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_65 (Conv2D)           (None, 26, 26, 77)        770       
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 24, 24, 117)       81198     
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 22, 22, 112)       118048    
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 20, 20, 52)        52468     
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 18, 18, 102)       47838     
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 16, 16, 122)       112118    
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 14, 14, 82)      

최고의 하이퍼 파라미터만 뽑았기 때문에 아직 모델학습이 되지 않았습니다!
이번에 epoch을 5번정도 주어서 모델학습을 진행합니다!

만일 여러분들이 무거운 모델을 돌릴 경우 하이퍼파라미터 튜닝작업이 매우 느려질 수 있습니다.
그때의 경우 하이퍼파라미터 튜닝할때 epoch을 3-4정도로 작게 준 다음 최고의 하이퍼파라미터를 뽑아낸 다음
본격적인 모델학습때 epoch을 넉넉하게 주는 것도 방법입니다!

In [11]:
model.fit(X_train, y_train, batch_size=32, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 32s 20ms/step - loss: 0.2631 - accuracy: 0.9217
Epoch 2/5
1500/1500 [==============================] - 30s 20ms/step - loss: 0.0894 - accuracy: 0.9741
Epoch 3/5
1500/1500 [==============================] - 30s 20ms/step - loss: 0.0708 - accuracy: 0.9792
Epoch 4/5
1500/1500 [==============================] - 30s 20ms/step - loss: 0.0605 - accuracy: 0.9817
Epoch 5/5
1500/1500 [==============================] - 26s 17ms/step - loss: 0.0499 - accuracy: 0.9847


모델 평가를 진행해볼까요?

In [12]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 2s 4ms/step - loss: 0.0436 - accuracy: 0.9862


[0.04359979182481766, 0.9861999750137329]

In [ ]:
이제 모델을 저장할 차례입니다.

우리가 이전까지 자주 사용한 저장방법은 HDF5파일 (.h5)로 저장하는 방법이었습니다.

HDF파일로 저장하는 방식은 이전 Keras에서 모델을 저장하는 방식이었으나 사실 이 방법은 TensorFlow나 Keras에서 그다지 선호하지 않는 저장방식입니다.

TensorFlow의 경우 공식적으로 지원하는 모델 저장방식은 SavedModel입니다.

SavedModel은 .h5파일처럼 모델의 가중치와 모델을 전부 하나의 파일로 관리하는 방식이 아닌 모델, 가중치를 따로 구분해서 저장하는 방식입니다.

SavedModel은 크게 3가지로 구성되어 있습니다.

saved_model.pb : pb는 프로토콜 버퍼를 의미하며 해당 파일은 내보낸 모델 그래프 구조를 포함하고 있습니다.
variables : 내보낸 변수값이 있는 이진 파일과 내보낸 모델 그래프에 해당하는 체크포인트를 포함하고 있습니다
assets : 내보낸 모델을 불러올 때 추가적인 파일이 필요한 경우 이 폴더에 파일이 생성됩니다.
이 방식으로 진행한다면 모델을 배포할 때 유리합니다.

Keras의 경우 .keras파일을 선호합니다. .keras파일은 .h5파일과 마찬가지로 가중치와 모델을 전부 하나의 파일로 관리합니다.

In [13]:
save_path = os.getenv('HOME') + '/aiffel/mlops/best_model/1'
fname = os.path.join(save_path, 'model')
model.save(fname)

INFO:tensorflow:Assets written to: /aiffel/aiffel/mlops/best_model/1/model/assets


13-7. 이제는 모델을 배포할 차례! : TFServing & TFLite

In [ ]:
모델을 만들었다면 이제는 배포를 진행해봐야죠!

배포를 진행하는 방법은 크게 2가지로 나눌 수 있습니다.

클라우드를 활용해서 모델을 배포하는 방식
경량화된 모델을 만들어서 휴대폰같은 디바이스에서도 모델이 실행되게 만드는 방식
TensorFlow는 첫번째 방식을 TFServing을 통해서 가능하게 만들며 2번째 방식은 TFLite방식으로 가능하게 만듭니다.
그렇다면 2개를 더 자세하게 알아볼까요?

TFServing
TFServing이란 텐서플로우 그래프를 배포할 수 있으며 표준화된 엔드포인트를 제공합니다. 또한 모델 및 버전관리가 가능하며 정책 기반으로 모델을 서비스할 수 있습니다.
또한 지연 시간이 최대한 짧게 만드는 고성능 처리량에서도 초점을 맞추고 있습니다.

TFServing을 하는 방식은 크게 2가지가 있습니다.

Docker를 활용한 배포
우분투 터미널을 활용한 배포
이번에 2개 모두 설명할 예정입니다.

주의사항 우분투 터미널 실습의 경우 실제 결과물이 나오려면 로컬에서 진행해야 합니다.
LMS에서 실행되지 않는 이유는 LMS 시스템에 들어가 있는 GPU클라우드도 Docker Image이며 쿠버네티스로 관리되고 있습니다.
그렇기에 WSL2와 Docker 환경세팅을 해주세요..!!

WSL2 설치 + 윈도우에서 Docker 설치하기
파일을 우분투 디렉토리로 옮기는 방법
macOS에서 Docker 설치하기
TFServing 우분투 터미널로 실습하기
우선 우분투 터미널 실습부터 진행하겠습니다!

클라우드 쉘을 열고 해당 스크립트를 넣어주세요!

docker run -p 8500:8500 \
			-p 8501:8501 \
			--mount type=bind,source=/tmp/models,target=/models/my_model
			-e MODEL_NAME=my_model \
			-e MODEL_BASE_PATH=/Users/komyeongji/Desktop/docker/model \
			-t tensorflow/serving

docker run  -p 8500:8500 \
			-p 8501:8501 \
			--mount type=bind,source=/tmp/models,target=/models/my_model
			-e MODEL_NAME=my_model \
			-e MODEL_BASE_PATH=/models/my_model \
			-t tensorflow/serving

In [ ]:
# docker run  -p 8500:8500 \
# 			-p 8501:8501 \
# 			--mount type=bind,source=/tmp/models,target=/Users/komyeongji/Desktop/docker/model
# 			-e MODEL_NAME=my_model \
# 			-e MODEL_BASE_PATH=/Users/komyeongji/Desktop/docker/model \
# 			-t tensorflow/serving

In [ ]:
# docker run tensorflow/serving -d -p 8500:8500 -p 8501:8501 \
#            --mount type=bind,source=/Users/komyeongji/Desktop/docker/1,target=/models \
#            -e MODEL_NAME=my_model \
#            -e MODEL_BASE_PATH=/models/model/1 \ # 수정
#            -t tensorflow/serving

/Users/komyeongji/Desktop/docker/model

In [15]:
load_path = os.getenv('HOME') + '/aiffel/mlops/best_model/1/model'
best_model = tf.keras.models.load_model(load_path)

In [16]:
best_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_65 (Conv2D)           (None, 26, 26, 87)        870       
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 24, 24, 87)        68208     
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 22, 22, 82)        64288     
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 20, 20, 62)        45818     
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 18, 18, 72)        40248     
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 16, 16, 77)        49973     
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 14, 14, 77)      

그렇다면 이제 tflite파일로 변환을 진행해보도록 하겠습니다!

변환을 진행할 떄에는 tf.lite.TFLiteConverter메소드를 활용하면 쉽게 바꿀 수 있습니다!

In [17]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp3t8kq7p0/assets


In [ ]:
tflite파일을 만들어서 우선 보관하도록 하겠습니다!

In [19]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [20]:
classify_lite = interpreter.get_signature_runner('serving_default')
classify_lite

NameError: name 'interpreter' is not defined